# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Frameworks</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

## Estudo de Caso - Geração Automática de Texto com GluonNLP

O GluonNLP fornece implementações de modelos de Deep Learning para Processamento de Linguagem Natural. Ele foi projetado para engenheiros, pesquisadores e estudantes criarem protótipos de ideias e produtos de pesquisa com base nesses modelos e de forma rápida. De fato, há pouca programação a ser feita. Está quase tudo pronto no GluonNLP. 

O GluonNLP traz os principais modelos de PLN prontos para uso com pouco esforço flexibilidade e alta usabilidade.

Vamos usar o GluonNLP para geração automática de texto.

Visite o site oficial aqui:

https://gluon-nlp.mxnet.io/

## Definição do Problema

Este Estudo de Caso demonstra como gerar texto usando um modelo de linguagem pré-treinado das duas maneiras a seguir:

- Com amostrador de sequência (sequence sampler)
- Com amostrador de busca por feixe (beam search sampler)

Variáveis a serem configuradas ao gerar sequências:

- V = tamanho do vocabulário
- T = comprimento da sequência
- V ^ T = o número de resultados possíveis para considerar uma sequência.

Dado um modelo de linguagem, podemos gerar sequências de acordo com a probabilidade de ocorrerem. A cada etapa do tempo, um modelo de linguagem prediz a probabilidade de cada palavra ocorrer, considerando o contexto das etapas anteriores. As saídas a qualquer momento podem ser qualquer palavra do vocabulário cujo tamanho é V e, portanto, o número de todos os resultados possíveis para uma sequência de comprimento T é, portanto, V ^ T.

Embora algumas vezes desejemos gerar sentenças de acordo com a probabilidade de ocorrência, outras vezes desejamos encontrar as sentenças que **têm maior probabilidade de ocorrer**. Isso é especialmente verdade no caso da tradução de idiomas, onde não queremos apenas ver uma tradução qualquer. Queremos a melhor tradução. Embora encontrar o resultado ideal rapidamente se torne intratável com o aumento do tempo, ainda existem muitas maneiras de gerar sequências razoavelmente boas. O GluonNLP fornece dois amostradores para gerar texto a partir de um modelo de linguagem: SequenceSampler e BeamSearchSampler. Usaremos ambos.

In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
# Instalamos o MXNET (não é necessário suporte a GPU)
!pip install -q mxnet

In [4]:
# Agora instalamos o GluonNLP
!pip install -q gluonnlp

In [5]:
# Imports
import numpy as np
import mxnet as mx
import gluonnlp as nlp

# Esse pacote foi baixado do repositório do GluonNLP e está sendo fornecido a você junto com este Jupyter Notebook
# https://github.com/dmlc/gluon-nlp
import text_generation.model

/Users/dmpm/opt/anaconda3/lib/python3.7/site-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

numpy    1.18.4
mxnet    1.6.0
gluonnlp 0.8.1
Data Science Academy


### Carregando o Modelo Pré-Treinado

In [7]:
# Vamos alterar o dispositivo para CPU (GPU não é necessário neste estudo de caso)
ctx = mx.cpu()

In [8]:
# Vamos importar o modelo pré-treinado para geração de texto
model, vocab = text_generation.model.get_model(name = 'gpt2_117m',
                                               dataset_name = 'openai_webtext',
                                               pretrained = True,
                                               ctx = ctx)

In [9]:
# Criamos então o tokenizador
tokenizer = nlp.data.GPT2BPETokenizer()

In [10]:
# E também o objeto para remover a tokenização (usaremos para mostrar o texto gerado)
detokenizer = nlp.data.GPT2BPEDetokenizer()

In [11]:
# Definimos o token de final de texto
eos_id = vocab[vocab.eos_token]
print(vocab.eos_token)

<|endoftext|>


### Sequence Sampler

Um SequenceSampler gera amostras da distribuição multinomial contextual produzida pelo modelo de linguagem a cada etapa do tempo. Podemos usar a opção de "temperatura" no SequenceSampler, que controla a "temperatura" da função softmax. Temperatura aqui nada mais é do que a intensidade, mas é assim que o parâmetro é chamado.

Para cada entrada igual, o Sequence Sampler pode amostrar várias sequências independentes de uma só vez. O número de sequências independentes a serem amostradas pode ser especificado através do argumento beam_size.

In [12]:
# Esta string será usada como ponto de partida para a geração de texto
bos_str = 'Deep learning and natural language processing'

In [13]:
# Adicionamos um espaço no início da string
if not bos_str.startswith(' '):
    bos_str = ' ' + bos_str

In [14]:
# Tokenizamos a string
bos_tokens = tokenizer(bos_str)

In [15]:
# Geramos o vocabulário com os tokens
bos_ids = vocab[bos_tokens]
print(bos_tokens)

['ĠDeep', 'Ġlearning', 'Ġand', 'Ġnatural', 'Ġlanguage', 'Ġprocessing']


Agora definimos o decoder.

In [16]:
# Classe para o decoder
class GPT2Decoder(text_generation.model.LMDecoder):
    def __call__(self, inputs, states):
        
        # Recebe os inputs
        inputs = inputs.expand_dims(axis = 1)
        
        # Gera as saídas
        out, new_states = self.net(inputs, states)
        
        # Reshape das saídas
        out = mx.nd.slice_axis(out, axis = 1, begin = 0, end = 1).reshape((inputs.shape[0], -1))
        
        return out, new_states

In [17]:
# Cria o objeto
decoder = GPT2Decoder(model)

E precisamos definir o estado inicial.

In [18]:
# Função para o estado inicial
def get_initial_input_state(decoder, bos_ids, temperature):
    
    # Inputs e estado inicial
    inputs, begin_states = decoder.net(mx.nd.array([bos_ids], dtype = np.int32, ctx = ctx), None)
    
    # Reshape dos inputs
    inputs = inputs[:, -1, :]
    
    # Probabilidades (observe o parâmetro de temperatura)
    smoothed_probs = (inputs / temperature).softmax(axis = 1)
    
    # Amostra multidimensional
    inputs = mx.nd.sample_multinomial(smoothed_probs, dtype = np.int32)
    
    return inputs, begin_states

In [19]:
# Hiperparâmetros do modelo
beam_size = 2
temperature = 0.97
num_results = 2
max_len = 256 - len(bos_tokens)

In [20]:
# Cria o sampler
sampler = nlp.model.SequenceSampler(beam_size = beam_size,
                                    decoder = decoder,
                                    eos_id = eos_id,
                                    max_length = max_len,
                                    temperature = temperature)

In [21]:
# Função para geração de texto
def generate(decoder, bos_ids, temperature, sampler, num_results, vocab):
    
    # Inputs e estado inicial
    inputs, begin_states = get_initial_input_state(decoder, bos_ids, temperature)
    
    # Amostras, escores e comprimentos válidos
    samples, scores, valid_lengths = sampler(inputs, begin_states)
    
    # Converte amostras, scores e comprimentos válidos para o formato numpy
    samples = samples[0].asnumpy()
    scores = scores[0].asnumpy()
    valid_lengths = valid_lengths[0].asnumpy()

    # Resultado
    print('\nResultado Gerado:\n')
    for i in range(num_results):
        
        # Gera os tokens (novo texto)
        generated_tokens = [vocab.idx_to_token[ele] for ele in samples[i][:valid_lengths[i]]]
        
        # Adiciona os tokens gerados ao texto inicial
        tokens = bos_tokens + generated_tokens[1:]
        
        # Desfaz a tokenização para mostrar o resultado no formato de texto
        print([detokenizer(tokens).strip(), scores[i]])

In [22]:
# Executa o gerador de texto
generate(decoder, bos_ids, temperature, sampler, num_results, vocab)


Resultado Gerado:

["Deep learning and natural language processing serious improvements over existing programming languages.\n\nNo framework or language developers have yet heard about the credentials of Cobolite's hardware manager but the potential of OpenCL/TODO for coding should seem unlimited.<|endoftext|>", -180.59767]
["Deep learning and natural language processing steps that make them easier to define than games or programs. Students with a BPU school require rigorous tutorial training that can allow them to code with their native language and to dig deeper into neural networks. The combination of software practice and instilled knowledge will allow instructors to develop the skills that will help them to highly perform their development tasks. Building a STEM projects lasting through formal and clinical work and into their career. Women's education has the basic undergraduate competencies. Opposing field based in STEM, applied... Learn mathematics. Midstream mentoring expands 

Texto gerado com sucesso. Vejamos se conseguimos melhorar a performance mudando o sampler.

### Beam Search Sampler

Para superar a complexidade exponencial na decodificação de sequência, a pesquisa por feixe faz uma decodificação mais intensa, mantendo as sequências que provavelmente são baseadas na probabilidade até a etapa de tempo atual. 

O tamanho desse subconjunto é chamado de tamanho do feixe (beam size). 

Suponha que o tamanho do feixe seja K e o tamanho do vocabulário de saída seja V. Ao selecionar os feixes a serem mantidos, o algoritmo Beam Search primeiro prediz todas as possíveis palavras sucessoras dos feixes K anteriores, cada um com V saídas possíveis. Isso se torna um total de caminhos K * V. Desses caminhos K * V, a pesquisa por feixe os classifica por sua pontuação, mantendo apenas os caminhos K principais.

O BeamSearchScorer é um HybridBlock simples que implementa a função de pontuação com penalidade de comprimento no artigo do Google NMT:

scores = (log_probs + scores) / length_penalty

length_penalty = (K + length)^alpha / (K + 1)^alpha

In [23]:
# Cria o scorer, que vai definir a intensidade da decodificação
scorer = nlp.model.BeamSearchScorer(alpha = 0, K = 5, from_logits = False)

In [24]:
# Cria o sampler
beam_sampler = nlp.model.BeamSearchSampler(beam_size = 3,
                                           decoder = decoder,
                                           eos_id = eos_id,
                                           scorer = scorer,
                                           max_length = max_len)

In [25]:
# Gera o texto
generate(decoder, bos_ids, temperature, beam_sampler, num_results, vocab)


Resultado Gerado:

['Deep learning and natural language processing\n\nThe study was published in the journal Proceedings of the National Academy of Sciences.<|endoftext|>', -13.526959]
['Deep learning and natural language processing\n\nThe study was published in the journal Proceedings of the National Academy of Sciences.\n\nExplore further: Researchers discover a new way to learn about the brain\n\n\nMore information: "A new way to learn about the brain: A new way to learn about the brain," Proceedings of the National Academy of Sciences, DOI: 10.10.10.10731701/pnas.1701221709617410<|endoftext|>', -91.53121]


Hummm...melhorou de forma considerável. Talvez um ajuste dos hiperparâmetros deixe o resultado ainda melhor, mas isso agora é com você.

# Fim